<a href="https://colab.research.google.com/github/37khalil/SVM-Implementation/blob/main/Car_SVM_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>